In [70]:
import pandas as pd
import numpy as np
import wrds
from fredapi import Fred

In [60]:
conn = wrds.Connection(wrds_username="nberzins")

Loading library list...
Done


In [61]:
conn.describe_table(library= "ff", table='fivefactors_daily')

Approximately 14836 rows in ff.fivefactors_daily.


,name,nullable,type,comment
0,date,True,DATE,Date
1,mktrf,True,"NUMERIC(8, 6)",Excess Return on the Market
2,smb,True,"NUMERIC(8, 6)",Small-Minus-Big Return
3,hml,True,"NUMERIC(8, 6)",High-Minus-Low Return
4,rmw,True,"NUMERIC(8, 6)",Robust Minus Weak Return
5,cma,True,"NUMERIC(8, 6)",Conservative Minus Aggressive Return
6,rf,True,"NUMERIC(7, 5)",Risk-Free Return Rate (One Month Treasury Bill...
7,umd,True,"NUMERIC(8, 6)",Momentum


In [19]:
# sp500 = conn.raw_sql("""
#                         SELECT a.*, b.date, b.ret
#                         FROM crsp.dsp500list as a, crsp.dsf as b
#                         WHERE a.permno=b.permno
#                             AND b.date >= a.start and b.date <= a.ending
#                             AND b.date >= '01/01/2000'
#                         ORDER BY date; 
#                         """, date_cols=['start', 'ending', 'date'])

In [18]:
# dse = conn.raw_sql("""
#                         SELECT comnam, namedt, nameendt,
#                         permno, shrcd, exchcd
#                         FROM crsp.dsenames
#                         """, date_cols=['namedt', 'nameendt'])

In [69]:
fama_five = conn.raw_sql("""
                            SELECT *
                            FROM ff.fivefactors_daily as ff
                            WHERE ff.date >= '01/01/2000'
                            """)

fama_five["date"] = pd.to_datetime(fama_five["date"], format='%Y-%m-%d')

fama_five.tail(5)

,date,mktrf,smb,hml,rmw,cma,rf,umd
6284,2024-12-24,0.0111,-0.0012,-0.0005,-0.0013,-0.0037,0.00017,0.0067
6285,2024-12-26,0.0002,0.0109,-0.0019,-0.0044,0.0035,0.00017,0.0001
6286,2024-12-27,-0.0117,-0.0044,0.0056,0.0041,0.0003,0.00017,-0.0088
6287,2024-12-30,-0.0109,0.0024,0.0074,0.0055,0.0014,0.00017,0.0006
6288,2024-12-31,-0.0046,0.0031,0.0071,0.0033,0.0,0.00017,-0.0107


In [103]:
fred_api_key = "62ce35723dc6bc5a66d7276925744d91"

fred = Fred(api_key = fred_api_key)
fed_funds = fred.get_series('FEDFUNDS', observation_start='2000-01-01')

In [104]:
ffer_df = fed_funds.to_frame(name="fed_funds_rate").reset_index()

ffer_df.describe()
ffer_df = ffer_df.rename(columns={"index": "date", "fed_funds_rate": "fed_funds_rate"})

ffer_df["date"] = pd.to_datetime(ffer_df["date"], format="%Y-%d-%m")
ffer_df["date"] = ffer_df["date"].dt.strftime('%Y-%m-%d')

ffer_df.head(35)

,date,fed_funds_rate
0,2000-01-01,5.45
1,2000-02-01,5.73
2,2000-03-01,5.85
3,2000-04-01,6.02
4,2000-05-01,6.27
5,2000-06-01,6.53
6,2000-07-01,6.54
7,2000-08-01,6.50
8,2000-09-01,6.52
9,2000-10-01,6.51


In [16]:
# sp500_full = pd.merge(sp500, dse, how='left', on='permno')

# sp500_full = sp500_full.loc[(sp500_full.date>=sp500_full.namedt) \
#                             & (sp500_full.date<=sp500_full.nameendt)]

# sp500_full.head()

In [46]:
# sp500_full.to_csv("sp500_full.csv", index = False)

sp500_full = pd.read_csv("sp500_full.csv")

In [47]:
#sp500_full = sp500_full[sp500_full["start"]]

sp500_full[sp500_full["comnam"].isin(['GENERAL ELECTRIC CO'])]

,permno,start,ending,date,ret,comnam,namedt,nameendt,shrcd,exchcd
114,12060,1957-03-01,2024-12-31,2000-01-03,-0.030695,GENERAL ELECTRIC CO,1968-01-02,2000-02-29,11,1
667,12060,1957-03-01,2024-12-31,2000-01-04,-0.038750,GENERAL ELECTRIC CO,1968-01-02,2000-02-29,11,1
1278,12060,1957-03-01,2024-12-31,2000-01-05,-0.005202,GENERAL ELECTRIC CO,1968-01-02,2000-02-29,11,1
1949,12060,1957-03-01,2024-12-31,2000-01-06,0.015577,GENERAL ELECTRIC CO,1968-01-02,2000-02-29,11,1
2327,12060,1957-03-01,2024-12-31,2000-01-07,0.038721,GENERAL ELECTRIC CO,1968-01-02,2000-02-29,11,1
...,...,...,...,...,...,...,...,...,...,...
3153152,12060,1957-03-01,2024-12-31,2024-12-24,0.014614,GENERAL ELECTRIC CO,2024-04-02,2024-12-31,11,1
3153733,12060,1957-03-01,2024-12-31,2024-12-26,0.003849,GENERAL ELECTRIC CO,2024-04-02,2024-12-31,11,1
3153811,12060,1957-03-01,2024-12-31,2024-12-27,-0.010398,GENERAL ELECTRIC CO,2024-04-02,2024-12-31,11,1
3154521,12060,1957-03-01,2024-12-31,2024-12-30,-0.011642,GENERAL ELECTRIC CO,2024-04-02,2024-12-31,11,1


In [48]:
# Date str --> Datetime conversion + ret --> log returns conversion

sp500_full["date"] = pd.to_datetime(sp500_full["date"], format='%Y-%m-%d')

sp500_full["ret"] = np.log(sp500_full["ret"] + 1)

sp500_full["ret"].sample(5)

204677     0.000000
3010219    0.002399
2265639   -0.000458
2325590   -0.002727
1007139   -0.017527
Name: ret, dtype: float64

In [49]:
sp500_dly = {
    name: group[["date", "ret"]].reset_index(drop=True)
    for name, group in sp500_full.groupby("comnam")
}

# sp500_dict["GENERAL ELECTRIC CO"]

In [107]:
sp500_wkly = {}

for name, df in sp500_dly.items():
    df = df.copy()
    df["date"] = pd.to_datetime(df["date"]) # double checking date time conversion
    df.set_index("date", inplace=True)

    weekly_df = df.resample("W-MON", label="left", closed="left")["ret"].sum().reset_index()

    sp500_wkly[name] = weekly_df

sp500_wkly["GENERAL ELECTRIC CO"]

,date,ret
0,2000-01-03,-0.022465
1,2000-01-10,-0.002068
2,2000-01-17,-0.046600
3,2000-01-24,-0.072843
4,2000-01-31,0.054902
...,...,...
1300,2024-12-02,-0.036789
1301,2024-12-09,-0.056047
1302,2024-12-16,0.014116
1303,2024-12-23,0.011751


In [105]:
sp500_monthly = {}

for name, df in sp500_dly.items():
    df = df.copy()
    df["date"] = pd.to_datetime(df["date"]) # double checking date time conversion
    df.set_index("date", inplace=True)

    monthly_df = df.resample("MS", label="left", closed="left")["ret"].sum().reset_index()

    sp500_monthly[name] = monthly_df

# sp500_monthly.items()

In [ ]:
#Need to Build Different Retun-Based Factors from Returns 

#e.g. Momentum, Volatility from past 30 days, market portfolio, banking/construction (responsive to IR), IR itself,
# others can be extracxted from French Fama ---- Shoot for 10 - 15 predictors for first iteration

# *Maybe find sentiment indices*

In [ ]:
#For next week -----------------------------

# Convert to weekly + Monthly data, too much noise in daily 
# Many large positions flip before market close to avoid sitting overnight 

#Focus on predictors and run standard models in ord

In [ ]:
# Next iteration would be to just include a ton of different predictors, 